# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$ (see also Problem 1 in hw3t). In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in hw4-theoretical.

### Question 2

To implement your reduction from Problem 1 in hw4t, you will work with some standard benchmark instances for min-cost flow included in the hw4p folder on canvas. The format of the data is described in the Info file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [33]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    
    def read_edges(infile, G):
        file = open(infile)
        node = 0
        for line in file:
            if 'p' in line:
                continue
            if 'a' in line:
                indx = line.index('a')
                line = line[(indx+2):]
                s1, s2, d, cap, cost = line.strip().split()
                G.add_edge(s1, s2, capacity = float(cap), weight = float(cost))
                if (s1, s2) in G.edges(): # add new edge for every multi-edge in graph
                    new_node = 'node' + str(node)
                    G.add_node(new_node)
                    G.add_edge(s1, new_node)
                    G.edges[s1, new_node]['capacity']=float(cap)
                    G.edges[s1, new_node]['weight']=0
                    G.add_edge(new_node, s2)
                    G.edges[new_node, s2]['capacity']=float(cap)
                    G.edges[new_node, s2]['weight']=float(cost)
                    G.remove_edge(s1,s2)
                    node += 1
                else:
                    G.add_edge(s1, s2, capacity = float(cap), weight = float(cost))
                
        return G
        
    def read_nodes(infile, G):
        for node in G.nodes():
            G.add_node(node)
            G.nodes[node]['demand'] = 0
        
        file = open(infile) #open file everytime need to read new one in
        for line in file:
            if 'p' in line:
                continue
            if 'n' in line:
                indx = line.index('n')
                line = line[(indx+2):]
                s, d = line.strip().split()
                G.add_node(s, demand=float(d))
                
        return G
    
    G = nx.DiGraph()
    G = read_edges(infile, G)
    G = read_nodes(infile, G)
    return G

In [74]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print("Correct value for _40 instance:", nx.flow.min_cost_flow_cost(G_40) == 52099553858)
print("Correct value for _6830 instance:", nx.flow.min_cost_flow_cost(G_6830) == 299390431788)
print("Correct value for _176280 instance:", nx.flow.min_cost_flow_cost(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [260]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    problem = pulp.LpProblem("Min Cost Flow LP Problem", pulp.LpMinimize)
    f = 0
    
    for i,j in G.edges():
        flow = pulp.LpVariable("flow"+str(f), lowBound=0, cat='Continuous')
        G.edges[i,j]['flow'] = flow
        problem += G.edges[i,j]['flow'] <= G.edges[i,j]['capacity']
        f += 1
    
    for node in G.nodes():
        in_flow = [G.edges[i,j]['flow'] for i,j in G.edges() if j == node]
        out_flow = [G.edges[i,j]['flow'] for i,j in G.edges() if i == node]
        problem += (pulp.lpSum(in_flow) - pulp.lpSum(out_flow) == G.nodes[node]['demand'])
        
    problem += pulp.lpSum(G.edges[i,j]['flow']*G.edges[i,j]['weight'] for i,j in G.edges())
    
    problem.solve()
    return pulp.value(problem.objective)

The following will check that the LP finds the same optimal values as previously.

In [261]:
print( "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858)
print( "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788)
print( "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
